# Sentiment Analysis dari Instagram comments calon Presiden 2024 sebelum dan sesudah deklrasi calon wakil presiden menggunakan Naive Bayes


## Tujuan : mengetahui perbedaan sentiment dari komentar instagram sebelum dan sesudah deklarasi calon wakil presiden


<ul>
    <li>
    Aurelius Ivan Wijaya (00000054769)
    </li>
    <li>
    Rajendra Abhinaya (00000060445)
    </li>
    <li>
    Maecyntha Irelynn Tantra (00000055038)
    </li>
    <li>
    Patricia theodora (00000054093)
    </li>
<ul>

# Data Collection

apa sih yang sebenernya kita cari?
* web scrapping algoritm untuk data primer (ivan)
* labeling (pat)
* data sekunder (mae)
* stopword library indonesia (abhi)
* cari jurnal referensi yang sudah, sebagai literature review (all, min 4 per person)

In [35]:
import pandas as pd
import numpy as np

In [36]:
# import dataset    
# [ Primary Dataset ]
# anies_before = pd.read_csv('./Dataset/Anies/anies_before.csv', on_bad_lines='skip')
# anies_after = pd.DataFrame(pd.read_csv('./Dataset/Anies/anies_after.csv', on_bad_lines="skip"))
ganjar_before = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_before.csv', on_bad_lines="skip")) # get only 100 data
ganjar_before = ganjar_before[:100]
ganjar_after = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_after.csv', on_bad_lines="skip"))
ganjar_after = ganjar_after[:100]
prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_before.csv', on_bad_lines="skip"))
prabowo_before = prabowo_before[:100]
prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_after.csv', on_bad_lines="skip"))
prabowo_after = prabowo_after[:100]
# prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_before.csv', on_bad_lines="skip"))
# prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_after.csv', on_bad_lines="skip"   ))
detik_anies_before = pd.read_csv('./Dataset/Detik/detik_anies_before.csv', on_bad_lines='skip')
detik_anies_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_anies_before.csv', on_bad_lines="skip"))
detik_ganjar_before = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_ganjar_before.csv', on_bad_lines="skip"))
detik_ganjar_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_ganjar_after.csv', on_bad_lines="skip"))
detik_prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_before.csv', on_bad_lines="skip"))
detik_prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_after.csv', on_bad_lines="skip"   ))
# [ Secondary Dataset ]
instagram_cyber_comments = pd.read_csv('./Dataset/dataset_komentar_instagram_cyberbullying.csv')
tweet_tv = pd.read_csv('./Dataset/dataset_tweet_sentimen_tayangan_tv.csv')
tweet_pilkada = pd.read_csv('./Dataset/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
tweet_opini_film = pd.read_csv('./Dataset/dataset_tweet_sentiment_opini_film.csv')
tweet_cellular = pd.read_csv('./Dataset/dataset_tweet_sentiment_cellular_service_provider.csv')
prastyo_sentiment = pd.read_csv('./Dataset/prastyo-sentiment_all.csv')
sentiment_twitter_indonesia = pd.read_csv('./Dataset/Indonesian Sentiment Twitter Dataset Labeled.csv', sep='\t')
# sentiment_shopee = pd.read_csv('./Dataset/dataset_shopee2.csv', on_bad_lines="skip")
import chardet

# Detect encoding
with open('./Dataset/dataset_shopee2.csv', 'rb') as f:
    result = chardet.detect(f.read())
# Use the detected encoding when reading the CSV file
sentiment_shopee = pd.read_csv('./Dataset/dataset_shopee2.csv', on_bad_lines="skip", encoding=result['encoding'])

In [37]:
import pandas as pd

def balance_dataset(data, label_column, max_samples=None):

    # Determine the minimum number of samples for any label
    min_samples = data[label_column].value_counts().min()

    # Randomly select min_samples samples for each label
    balanced_subset = pd.concat([group.sample(min_samples) for _, group in data.groupby(label_column)])

    # Shuffle the balanced subset to mix the labels
    balanced_subset = balanced_subset.sample(frac=1).reset_index(drop=True)

    # Optionally only keep a maximum number of samples
    if max_samples is not None:
        balanced_subset = balanced_subset[:max_samples]
    
    return balanced_subset

## Data Integration

In [38]:
# 1: positive, 0: negative 

# data integration
# change label name to 'comments'
instagram_cyber_comments.rename(columns={'Instagram Comment Text': 'comments'}, inplace=True)
# mapping sentiment
instagram_cyber_comments['Sentiment'] = instagram_cyber_comments['Sentiment'].map({'positive': 1, 'negative': 0})
instagram_cyber_comments['label'] = instagram_cyber_comments['Sentiment'].astype(int)
# drop unused columns
instagram_cyber_comments.drop(columns=['Id', 'Sentiment'], inplace=True)
# display(instagram_cyber_comments.head())
#change data type to string to ensure all data type is string
instagram_cyber_comments['comments'] = instagram_cyber_comments['comments'].astype(str)

# tweet_tv
# change label name to 'comments'
tweet_tv.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_tv['Sentiment'] = tweet_tv['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_tv['label'] = tweet_tv['Sentiment'].astype(int)
tweet_tv.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_tv.drop(columns=['Id', 'Sentiment', "Jumlah Retweet", "Acara TV"], inplace=True)
#change data type to string to ensure all data type is string
tweet_tv['comments'] = tweet_tv['comments'].astype(str)

# tweet_pilkada
# tweet_pilkada add new column 'label'
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
tweet_pilkada['Sentiment'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].astype(int)
tweet_pilkada = tweet_pilkada[['Sentiment', 'Text Tweet']]
tweet_pilkada.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_pilkada.rename(columns={'Sentiment': 'label'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_pilkada['comments'] = tweet_pilkada['comments'].astype(str)

# tweet_opini_film
# change label name to 'comments'
tweet_opini_film.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_opini_film['Sentiment'] = tweet_opini_film['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_opini_film['label'] = tweet_opini_film['Sentiment'].astype(int)
tweet_opini_film.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_opini_film.rename(columns={'Text Tweet': 'comments'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_opini_film['comments'] = tweet_opini_film['comments'].astype(str)

# tweet_cellular
# change label name to 'comments'
tweet_cellular.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_cellular['Sentiment'] = tweet_cellular['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_cellular['label'] = tweet_cellular['Sentiment'].astype(int)
tweet_cellular.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_cellular.rename(columns={'Text Tweet': 'comments'}, inplace=True)

# prastyo_sentiment
map = {'pos': 1, 'neg': 0, 'neu': 1}
prastyo_sentiment['label'] = prastyo_sentiment['label'].map(map)


# sentiment twitter indonesia
# change label name to 'comments'
sentiment_twitter_indonesia.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
# sentiment_twitter_indonesia['sentimen'] = sentiment_twitter_indonesia['sentimen'].map({'positive': 1, 'negative': 0})
sentiment_twitter_indonesia['label'] = sentiment_twitter_indonesia['sentimen'].astype(int)


# shoppe
# change label name to 'comments'
sentiment_shopee.rename(columns={'Review': 'comments'}, inplace=True)
# mapping sentiment
sentiment_shopee['SENTIMEN'] = sentiment_shopee['SENTIMEN'].map({'POSITIF': 1, 'NEGATIF': 0})
sentiment_shopee['label'] = sentiment_shopee['SENTIMEN'].astype(int)
sentiment_shopee.drop(columns=['SENTIMEN'], inplace=True)


detik_anies_after = balance_dataset(detik_anies_after, 'label')
detik_anies_before = balance_dataset(detik_anies_before, 'label')
# detik_ganjar_before = balance_dataset(detik_ganjar_before, 'label', max_samples=50)
# detik_ganjar_after = balance_dataset(detik_ganjar_after, 'label', max_samples=50)

secondary_dataset = pd.concat([
    # anies_before, # if we train with anies_before, the accuracy will be decrease
    # anies_after,
    # detik_anies_after,
    detik_anies_before,
    detik_ganjar_after,
    detik_ganjar_before,
    # sentiment_twitter_indonesia,
    # tweet_opini_film
    tweet_pilkada,
    instagram_cyber_comments
    ], ignore_index=True)
# only pick comments and label column
secondary_dataset = secondary_dataset[['comments', 'label']]
#change data type to string to ensure all data type is string
secondary_dataset['comments'] = secondary_dataset['comments'].astype(str)
# if there is column 'Unnamed: 0' in secondary dataset, drop it
if 'Unnamed: 0' in secondary_dataset.columns:
    secondary_dataset.drop(columns=['Unnamed: 0'], inplace=True)

# Pre-Proccessing

## Data Cleaning

### Handle Missing Value

In [39]:
display(secondary_dataset.info())
display(secondary_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1608 entries, 0 to 1607
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   comments  1608 non-null   object 
 1   label     1608 non-null   float64
dtypes: float64(1), object(1)
memory usage: 25.3+ KB


None

,comments,label
0,apa yg omongan loe udah jalan waktu jadi gaben...,0.0
1,"Jika jalan hidupmu sulit n kacau, coba rubah n...",0.0
2,Sorat sorot sorat sorot.. koyo²o iso mbenahi a...,0.0
3,Ente emang paling pinter mengolah kata,0.0
4,Woi dp nol apartemen lu gimana kabare,0.0


In [40]:
def handleMissingValue(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[df['comments'] != '']
    # remove 'NaN' value
    if('NaN' in df['comments']):
        df = df[df['comments'] != 'NaN']
    df = df[df['comments'].notna()]
    df = df.reset_index(drop=True)
    df = df[['comments']] 
    return df
secondary_dataset['comments'] = handleMissingValue(secondary_dataset)
display(secondary_dataset.info())
display(secondary_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1608 entries, 0 to 1607
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   comments  1600 non-null   object 
 1   label     1608 non-null   float64
dtypes: float64(1), object(1)
memory usage: 25.3+ KB


None

,comments,label
0,apa yg omongan loe udah jalan waktu jadi gaben...,0.0
1,"Jika jalan hidupmu sulit n kacau, coba rubah n...",0.0
2,Sorat sorot sorat sorot.. koyo²o iso mbenahi a...,0.0
3,Ente emang paling pinter mengolah kata,0.0
4,Woi dp nol apartemen lu gimana kabare,0.0


In [41]:
print(secondary_dataset.isnull().sum())
# delete duplicate data and missing value and null value
secondary_dataset = secondary_dataset.dropna()
secondary_dataset = secondary_dataset.drop_duplicates()
secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)
# secondary_dataset = secondary_dataset[['comments']]
print(secondary_dataset.isnull().sum())

display(secondary_dataset)

comments    8
label       0
dtype: int64
comments    0
label       0
dtype: int64


,comments,label
0,apa yg omongan loe udah jalan waktu jadi gaben...,0.0
1,"Jika jalan hidupmu sulit n kacau, coba rubah n...",0.0
2,Sorat sorot sorat sorot.. koyo²o iso mbenahi a...,0.0
3,Ente emang paling pinter mengolah kata,0.0
4,Woi dp nol apartemen lu gimana kabare,0.0
...,...,...
1595,Bangga sama suami yg selalu ingat istri disela...,1.0
1596,Apaoun pekerjaannya yg penting halal u tuk men...,1.0
1597,Gojek itu mayoritas pegangguran yang lama gak ...,1.0
1598,<USERNAME> aslinya cantik dan ayu loh mbak kr...,1.0


### Text Cleaning

In [42]:
import string
import re
import pandas as pd

# Precompile regex patterns
re_long_word = re.compile(r'\b\w{20,}\b')
re_non_ascii = re.compile(r'[^\x00-\x7F]+')
# re_punctuation = re.compile(f"[{re.escape(string.punctuation)}]")
re_single_char = re.compile(r'\s+[a-zA-Z]\s+')
re_numbers = re.compile(r'\d+')
re_tags = re.compile("&lt;/?.*?&gt;")
re_special_chars_digits = re.compile("(\\d|\\W)+")
re_repeated_chars = re.compile(r'(.)\1+')

import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def clean_text(comment):
    comment = str(comment)
    comment = comment.lower() # Case Folding to lowercase

    # Remove non-ascii words and characters
    comment = re_non_ascii.sub(' ', comment) # remove non-ascii characters

    # Remove all links
    comment = re.sub(r"http\S+", "", comment) # remove all links

    # Delete all emoticons from text
    comment = re.sub(':[^ ]+:', '', comment) # remove emoticons
    comment = remove_emojis(comment)
    # remove all word that start with < and end with >
    comment = re.sub('<[^>]*>', '', comment)
    # Remove long words
    comment = re_long_word.sub('', comment) # remove long words

    # if length of comment is more than 7 and ended with "ny" convert it to "nya"
    if(len(comment) > 7 and comment[-2:] == 'ny'):
        comment = comment[:-2] + 'nya'
    # Remove single characters
    comment = re_single_char.sub(' ', comment) # remove single characters

    # Remove numbers
    comment = re_numbers.sub('', comment)

    # Remove word that contains tags
    comment = re_tags.sub('', comment)

    # Remove special characters and digits
    comment = re_special_chars_digits.sub(" ", comment) 

    # If a letter is repeated more than 2 times, replace it with 1 time
    comment = re_repeated_chars.sub(r'\1\1', comment)

    # Remove extra whitespaces
    comment = re.sub(r'\s+', ' ', comment)

    return comment.strip()

def remove_punct(data):
    data['comments'] = data['comments'].apply(clean_text)
    return data

In [43]:
# Call the function
secondary_dataset.to_csv('./dirty_data.csv', index=False)
cleaned_data = remove_punct(secondary_dataset)
secondary_dataset['comments'] = cleaned_data['comments']

secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)

# secondary_dataset = secondary_dataset[['comments']]
print(secondary_dataset.isnull().sum())
# Save to CSV

comments    0
label       0
dtype: int64


In [44]:
display(secondary_dataset)

,comments,label
0,apa yg omongan loe udah jalan waktu jadi gaben...,0.0
1,jika jalan hidupmu sulit kacau coba rubah nama...,0.0
2,sorat sorot sorat sorot koyo iso mbenahi ae nd...,0.0
3,ente emang paling pinter mengolah kata,0.0
4,woi dp nol apartemen lu gimana kabare,0.0
...,...,...
1583,bangga sama suami yg selalu ingat istri disela...,1.0
1584,apaoun pekerjaannya yg penting halal tuk menaf...,1.0
1585,gojek itu mayoritas pegangguran yang lama gak ...,1.0
1586,aslinya cantik dan ayu loh mbak krn aku sudah ...,1.0


### Stopword Removal

In [45]:
import re

display(secondary_dataset)
# Stopword removal

stopword_bahasa = pd.read_csv('./Dataset/TextNormalization/stopwordbahasa.csv', header=None)

def stopword_removal(data):
    for i in range(len(data)):
        data = data.replace(stopword_bahasa[0][i], '')
    return data

,comments,label
0,apa yg omongan loe udah jalan waktu jadi gaben...,0.0
1,jika jalan hidupmu sulit kacau coba rubah nama...,0.0
2,sorat sorot sorat sorot koyo iso mbenahi ae nd...,0.0
3,ente emang paling pinter mengolah kata,0.0
4,woi dp nol apartemen lu gimana kabare,0.0
...,...,...
1583,bangga sama suami yg selalu ingat istri disela...,1.0
1584,apaoun pekerjaannya yg penting halal tuk menaf...,1.0
1585,gojek itu mayoritas pegangguran yang lama gak ...,1.0
1586,aslinya cantik dan ayu loh mbak krn aku sudah ...,1.0


In [46]:

# Call the function
# cleaned_data = stopword_removal(secondary_dataset)
# convert comments to string
# secondary_dataset['comments'] = secondary_dataset['comments'].astype(str)

#check null value
secondary_dataset['comments'] = secondary_dataset['comments'].apply(stopword_removal)
# secondary_dataset['comments'] = secondary_dataset['comments'].apply(lambda x: stopword_removal(x))
display(secondary_dataset)

# handle null, missing value, and duplicate data
secondary_dataset = secondary_dataset.dropna()
secondary_dataset = secondary_dataset.drop_duplicates()
secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)
# secondary_dataset = secondary_dataset[['comments']]
display(secondary_dataset)

,comments,label
0,yg omongan loe udah jalan waktu ji gabener dk...,0.0
1,jika jalan hidupmu sulit kacau coba rubah nama...,0.0
2,sorat sorot sorat sorot koyo iso mbenahi ae nd...,0.0
3,ente emang paling pinter mengolah kata,0.0
4,woi dp nol rtemen lu gimana kabare,0.0
...,...,...
1583,bangga sama suami yg selalu ingat istri disela...,1.0
1584,oun pekerjaannya yg penting halal tuk menafkah...,1.0
1585,gojek itu mayoritas pegangguran yang lama gak ...,1.0
1586,aslinya cantik ayu loh mbak krn sudah bertem...,1.0


,comments,label
0,yg omongan loe udah jalan waktu ji gabener dk...,0.0
1,jika jalan hidupmu sulit kacau coba rubah nama...,0.0
2,sorat sorot sorat sorot koyo iso mbenahi ae nd...,0.0
3,ente emang paling pinter mengolah kata,0.0
4,woi dp nol rtemen lu gimana kabare,0.0
...,...,...
1578,bangga sama suami yg selalu ingat istri disela...,1.0
1579,oun pekerjaannya yg penting halal tuk menafkah...,1.0
1580,gojek itu mayoritas pegangguran yang lama gak ...,1.0
1581,aslinya cantik ayu loh mbak krn sudah bertem...,1.0


### Tokenization

In [47]:
# Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
secondary_dataset['comments'] = secondary_dataset['comments'].apply(word_tokenize)

display(secondary_dataset)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aurel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,comments,label
0,"[yg, omongan, loe, udah, jalan, waktu, ji, gab...",0.0
1,"[jika, jalan, hidupmu, sulit, kacau, coba, rub...",0.0
2,"[sorat, sorot, sorat, sorot, koyo, iso, mbenah...",0.0
3,"[ente, emang, paling, pinter, mengolah, kata]",0.0
4,"[woi, dp, nol, rtemen, lu, gimana, kabare]",0.0
...,...,...
1578,"[bangga, sama, suami, yg, selalu, ingat, istri...",1.0
1579,"[oun, pekerjaannya, yg, penting, halal, tuk, m...",1.0
1580,"[gojek, itu, mayoritas, pegangguran, yang, lam...",1.0
1581,"[aslinya, cantik, ayu, loh, mbak, krn, sudah, ...",1.0


### Text Normalization

In [48]:
# Text Normalization / Noise Removal
indo_slang_word = pd.read_csv("./Dataset/TextNormalization/lexicontest.csv")

display(indo_slang_word.shape)
display(indo_slang_word.head())

(5045, 2)

,slang,formal
0,22nya,dua-duanya
1,aaaaaaakkk,ah
2,aaaaahhhh,ah
3,aaaammmiiiinnnn,amin
4,aaahh,ah


In [49]:
# # def replace_slang_word(doc,slang_word):
# #     for index in  range(0,len(doc)-1):
# #         index_slang = slang_word.slang==doc[index]
# #         formal = list(set(slang_word[index_slang].formal))
# #         # replace slang word with formal word
# #         if len(formal)>0:
# #             doc[index] = formal[0]
        
# #     return doc
# def replace_slang_word(doc, slang_word):
#     for index in range(len(doc)):
#         slang_match = slang_word[slang_word['slang'].isin([doc[index]])]
#         if not slang_match.empty:
#             formal = list(set(slang_match['formal']))
#             if len(formal) > 0:
#                 doc[index] = formal[0]
#     return doc

# prevent word to cut in the middle of word for example: 'kemarin' -> 'kemarin' not 'kemar' 'in'

def replace_slang_word(doc, slang_word):
    for index in range(len(doc)):
        slang_match = slang_word[slang_word['slang'].isin([doc[index]])]
        if not slang_match.empty:
            formal = list(set(slang_match['formal']))
            if len(formal) > 0:
                doc[index] = formal[0]
    return doc

In [50]:
print(secondary_dataset)
# convert data frame to array
# secondary_dataset['comments'] = secondary_dataset['comments'].apply(lambda x: np.array(x))
normal_data = secondary_dataset
normal_data = secondary_dataset['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
# append data to secondary_dataset
print(secondary_dataset)

#testing
testing = secondary_dataset['comments'][0]
print(testing)
testing = replace_slang_word(testing, indo_slang_word)

                                               comments  label
0     [yg, omongan, loe, udah, jalan, waktu, ji, gab...    0.0
1     [jika, jalan, hidupmu, sulit, kacau, coba, rub...    0.0
2     [sorat, sorot, sorat, sorot, koyo, iso, mbenah...    0.0
3         [ente, emang, paling, pinter, mengolah, kata]    0.0
4            [woi, dp, nol, rtemen, lu, gimana, kabare]    0.0
...                                                 ...    ...
1578  [bangga, sama, suami, yg, selalu, ingat, istri...    1.0
1579  [oun, pekerjaannya, yg, penting, halal, tuk, m...    1.0
1580  [gojek, itu, mayoritas, pegangguran, yang, lam...    1.0
1581  [aslinya, cantik, ayu, loh, mbak, krn, sudah, ...    1.0
1582  [suami, saya, seumuran, sama, saya, mba, malah...    1.0

[1583 rows x 2 columns]
                                               comments  label
0     [yang, omongan, lo, sudah, jalan, waktu, ji, g...    0.0
1     [jika, jalan, hidupmu, sulit, kacau, coba, rub...    0.0
2     [sorat, sorot, sorat, so

### Stemming

In [51]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [52]:
def lemma_indo(doc):
    return [stemmer.stem(word) for word in doc]

In [53]:
# stemming
display(secondary_dataset['comments'][0])
stemmed_data = secondary_dataset['comments'].apply(lemma_indo)
# append stemmed data to secondary dataset, don't forget to reset index
secondary_dataset['comments'] = stemmed_data
secondary_dataset = secondary_dataset.reset_index(drop=True)
display(secondary_dataset)

['yang',
 'omongan',
 'kamu ',
 'sudah',
 'jalan',
 'waktu',
 'ji',
 'gabener',
 'dki',
 'bro']

,comments,label
0,"[yang, omong, kamu, sudah, jalan, waktu, ji, g...",0.0
1,"[jika, jalan, hidup, sulit, kacau, coba, rubah...",0.0
2,"[sorat, sorot, sorat, sorot, koyo, iso, mbenah...",0.0
3,"[anda, memang, paling, pintar, olah, kata]",0.0
4,"[woi, dp, nol, rtemen, kamu, bagaimana, kabare]",0.0
...,...,...
1578,"[bangga, sama, suami, yang, selalu, ingat, ist...",1.0
1579,"[oun, kerja, yang, penting, halal, untuk, nafk...",1.0
1580,"[gojek, itu, mayoritas, pegangguran, yang, lam...",1.0
1581,"[asli, cantik, ayu, loh, mbak, karena, sudah, ...",1.0


## Synthetic Minority Oversampling Technique (SMOTE)

* this one need research (ivan)
* Smote adalah sebuah tehnik yang digunakan terhadap data yang tidak seimbang

In [54]:
# Smote
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# clean empty row
# Instantiate SMOTE
smote = SMOTE(
    sampling_strategy='minority',
    random_state=42
)
# Assuming 'comments' is the name of the text column
comments = secondary_dataset['comments']
# Assuming 'label' is name of the target column
y = secondary_dataset['label']


# vectorizer tokenized comments
# vectorizer = CountVectorizer()
# TF-IDF vectorization
vectorizer = TfidfVectorizer()

# vectorizer = CountVectorizer()
# X_tfidf = vectorizer.fit_transform(comments)
joined_comments = pd.DataFrame(comments.apply(lambda x: ' '.join(x)))
# display(joined_comments)
# X_tfidf = vectorizer.fit_transform(joined_comments) # if we want to vectorize all comments in one row
# fit vectorizer
vectorizer.fit(joined_comments['comments'])
# transform comments
X_tfidf = vectorizer.transform(joined_comments['comments'])
# vectorizer.fit(comments)
# X_tfidf = vectorizer.transform(comments)
# # display(X_tfidf)
# # display(X_tfidf.shape)
# Apply SMOTE to data
X_smote, y_smote = smote.fit_resample(X_tfidf, y)

In [55]:
# without SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
# random forest
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# SVM
from sklearn.svm import SVC

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Instantiate Multinomial Naive Bayes classifier
classifier = MultinomialNB()
# Logistic Regression
classifier = LogisticRegression(random_state=42)
# Fit the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

Accuracy: 0.69
Classification Report:
               precision    recall  f1-score   support

         0.0       0.74      0.64      0.69       171
         1.0       0.64      0.74      0.69       146

    accuracy                           0.69       317
   macro avg       0.69      0.69      0.69       317
weighted avg       0.70      0.69      0.69       317



In [56]:
# with SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
# random forest
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Instantiate Multinomial Naive Bayes classifier
classifier = MultinomialNB(alpha=0.1)

# Fit the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)


Accuracy: 0.76
Classification Report:
               precision    recall  f1-score   support

         0.0       0.79      0.70      0.74       163
         1.0       0.73      0.81      0.77       161

    accuracy                           0.76       324
   macro avg       0.76      0.76      0.76       324
weighted avg       0.76      0.76      0.76       324



## Grid Search for Hyperparameter Tuning

In [57]:
from sklearn.model_selection import GridSearchCV
# import kfold
from sklearn.model_selection import KFold
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# All naive bayes
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# Create k-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter tuning for combination of alpha and fit_prior
model_param_grid = {
    'alpha': [0.01, 0.05,0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 100.0],
    'fit_prior': [True, False],
    'class_prior': [None, [0.25, 0.75], [0.5, 0.5], [0.75, 0.25]]
    }
# model_param_grid = {
#     'alpha': [0.01, 0.05,0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 100.0],
#     'fit_prior': [True, False],
#     }
# for gaussian naive bayes

model_grid_search = GridSearchCV(MultinomialNB(), model_param_grid, cv=kfold)
model_grid_search.fit(X_smote, y_smote)
# Print the best parameters and corresponding accuracy
print("\nBest parameters for MultinomialNB:", model_grid_search.best_params_)
print("Best accuracy for MultinomialNB:", model_grid_search.best_score_)

best_alpha = model_grid_search.best_params_['alpha']
best_fit_prior = model_grid_search.best_estimator_.fit_prior

display(best_alpha)
display(best_fit_prior)


Best parameters for MultinomialNB: {'alpha': 0.1, 'class_prior': None, 'fit_prior': False}
Best accuracy for MultinomialNB: 0.7657760960134542


0.1

False

## K-Fold Cross Validation

* this one need research (abhi)

In [58]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
# all forest algorithm
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier

# # Define 10-fold cross-validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# # Instantiate classifier 
classifier = MultinomialNB(alpha=best_alpha, fit_prior=best_fit_prior)

cv_results = cross_val_score(classifier, X_smote, y_smote, cv=kfold)
# # Print the results for each fold
best_fold = 0
best_accuracy = 0

for i, accuracy in enumerate(cv_results):
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_fold = i
        print(f"Fold {i + 1}: Accuracy = {accuracy} <== BEST RESULT")
    else:
        print(f"Fold {i + 1}: Accuracy = {accuracy}")

# from best fold, we will get the best model
# Instantiate classifier
# Print the mean and standard deviation of the cross-validation results
print(f"\nMean Accuracy: {cv_results.mean()}")
print(f"Standard Deviation: {cv_results.std()}")
# # After k-folds, now we will use the best model to predict the data

Fold 1: Accuracy = 0.7592592592592593 <== BEST RESULT
Fold 2: Accuracy = 0.7283950617283951
Fold 3: Accuracy = 0.8024691358024691 <== BEST RESULT
Fold 4: Accuracy = 0.6975308641975309
Fold 5: Accuracy = 0.7283950617283951
Fold 6: Accuracy = 0.7716049382716049
Fold 7: Accuracy = 0.7716049382716049
Fold 8: Accuracy = 0.7222222222222222
Fold 9: Accuracy = 0.8012422360248447
Fold 10: Accuracy = 0.7763975155279503

Mean Accuracy: 0.7559121233034276
Standard Deviation: 0.03347165402877794


## Workload Distribution?

# Modeling

<!-- * kemungkinan Binomial Naive Bayes -->

In [59]:
# train test split
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
# secondary_dataset = secondary_dataset.reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Instantiate classifier
classifier = MultinomialNB(alpha=best_alpha, fit_prior=best_fit_prior)

# Fit the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
display(X_smote)

# Contoh vektorisasi data baru
# new_data = ["deklarasi tukang tikung"]
# new_data_vectorized = vectorizer.transform(new_data)

# Prediksi label data baru
# new_data_pred = classifier.predict(new_data_vectorized)
# print(new_data_pred)

Accuracy: 0.76


<1618x4394 sparse matrix of type '<class 'numpy.float64'>'
	with 21752 stored elements in Compressed Sparse Row format>

In [60]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# Load the dataset

classifier.fit(X_smote, y_smote)

def prediction(data):    
    tokenized = []
    for i in range(0, len(data)):
        # Check if the value is NaN or not a string
        if pd.notna(data[i]) and isinstance(data[i], str):
            tokenized.extend(word_tokenize(data[i]))
    # Transform using the same vectorizer used during training
    text = vectorizer.transform([" ".join(tokenized)]).toarray()
    # return classifier.predict(text)
    return classifier.predict(text)

In [61]:
import pandas as pd

def process_and_save_data(X, prediction_function, output_path='./temp.csv'):
    pos = 0
    neg = 0
    label = []
    predictionLabel = []

    for i in range(0, len(X)):
        text = [X['comments'][i]]
        pred = prediction_function(text)
        predictionLabel.append(pred[0])
        if pred == 1:
            pos += 1
        else:
            neg += 1
        label.append(pred[0])

    X['label'] = label

    # Save the DataFrame to a CSV file
    X.to_csv(output_path, index=False)
    print("Positive:", pos)
    print("Negative:", neg)
    # print percentage of positive and negative
    print("Positive percentage:", pos/len(X)*100)
    print("Negative percentage:", neg/len(X)*100)
    return pos, neg, predictionLabel

In [62]:
# Assuming 'comments' is the text data in your dataset
# X = pd.read_csv('./Dataset/Detik/', on_bad_lines='skip')
X = detik_anies_after
# clean data
X = X.dropna()
X = remove_punct(X)
# norm
X['comments'] = X['comments'].apply(word_tokenize)
X['comments'] = X['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
X['comments'] = X['comments'].apply(lemma_indo)
# remove stopword

# convert back to string
X['comments'] = X['comments'].apply(lambda x: ' '.join(x))
# X['comments'] = X['comments'].apply(stopword_removal)
print(X['label'].value_counts())
X['comments'] = X['comments'].astype(str)
true_label = X['label']
pos, neg, predictionLabel = process_and_save_data(X, prediction, './temp.csv')

print("\n")
from sklearn.metrics import classification_report
print(classification_report(true_label, predictionLabel))

label
0.0    51
1.0    51
Name: count, dtype: int64
Positive: 57
Negative: 45
Positive percentage: 55.88235294117647
Negative percentage: 44.11764705882353


              precision    recall  f1-score   support

         0.0       0.84      0.75      0.79        51
         1.0       0.77      0.86      0.81        51

    accuracy                           0.80       102
   macro avg       0.81      0.80      0.80       102
weighted avg       0.81      0.80      0.80       102



In [70]:
# Assuming 'comments' is the text data in your dataset
# X = pd.read_csv('./Dataset/Detik/', on_bad_lines='skip')
# X = 
# clean data
X = X.dropna()
X = remove_punct(X)
# norm
X['comments'] = X['comments'].apply(word_tokenize)
X['comments'] = X['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
X['comments'] = X['comments'].apply(lemma_indo)
# remove stopword

# convert back to string
X['comments'] = X['comments'].apply(lambda x: ' '.join(x))
# X['comments'] = X['comments'].apply(stopword_removal)
print(X['label'].value_counts())
X['comments'] = X['comments'].astype(str)
true_label = X['label']
pos, neg, predictionLabel = process_and_save_data(X, prediction, './temp.csv')

print("\n")
from sklearn.metrics import classification_report
print(classification_report(true_label, predictionLabel))

label
0    161
1    139
Name: count, dtype: int64
Positive: 146
Negative: 154
Positive percentage: 48.66666666666667
Negative percentage: 51.33333333333333


              precision    recall  f1-score   support

           0       0.63      0.60      0.62       161
           1       0.56      0.59      0.58       139

    accuracy                           0.60       300
   macro avg       0.60      0.60      0.60       300
weighted avg       0.60      0.60      0.60       300



# Model Evaluation

* this one need research (pat)

# Analysis the data from primary data

## Modal Comprarison

# Conclusion